In [54]:
# Config (update these variables based on your GCP project and naming conventions)

GCP_PROJECT_ID           = 'dz-apps'        # Google Cloud Project ID
BQ_DATASET               = 'zdatasets'      # Name of BigQuery Dataset to create (if it does not exist)
BQ_MODEL_NAME            = 'logistic_model' # Name of BigQuery model that will be trained
GCS_BUCKET               = 'bqml_vertex'    # Google Cloud Storage Bucket to be created
GCS_LOCATION             = 'us-central1'    # Google Cloud Storage Location
VERTEX_LOCATION          = 'us-central1'
VERTEX_MODEL_NAME        = 'bqml-to-vertex'
VERTEX_SERVING_CONTAINER = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-3:latest' # https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers

----

## **BigQuery ML model deployment to Vertex AI**
References: 
* [Vertex/AI Platform SDK](https://github.com/googleapis/python-aiplatform)
* [BQML Exporting Models](https://cloud.google.com/bigquery-ml/docs/exporting-models#bq)
* [BQML Logistic Example](https://cloud.google.com/bigquery-ml/docs/logistic-regression-prediction)
* [BQML Model Types](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-create#model_type)
* [Codelab example that I modified for this notebook](https://codelabs.developers.google.com/codelabs/bqml-vertex-prediction)

In [44]:
!pip install google-cloud-aiplatform==1.0.1

     |████████████████████████████████| 1.8 MB 2.2 MB/s eta 0:00:01


In [46]:
from google.cloud import bigquery
from google.cloud import storage
from google.cloud import aiplatform

In [36]:
# Initialize BQ client
try:
    bq_client = bigquery.Client()
except Exception as e: 
    print('[ EXCEPTION ] {}'.format(e))

QUERY = f'''
CREATE OR REPLACE MODEL
  `{GCP_PROJECT_ID}.{BQ_DATASET}.{BQ_DATASET}` OPTIONS(model_type='LOGISTIC_REG',
    input_label_cols=['default_payment_next_month']) AS
SELECT
  limit_balance,
  sex,
  education_level,
  marital_status,
  age,
  pay_0,
  pay_2,
  pay_3,
  pay_4,
  pay_5,
  pay_6,
  bill_amt_1,
  bill_amt_2,
  bill_amt_3,
  bill_amt_4,
  bill_amt_5,
  bill_amt_6,
  pay_amt_1,
  pay_amt_2,
  pay_amt_3,
  pay_amt_4,
  pay_amt_5,
  pay_amt_6,
  default_payment_next_month
FROM
  `bigquery-public-data.ml_datasets.credit_card_default`
'''
query_job = bq_client.query(QUERY)
rows = query_job.result()

In [37]:
# Create Google Cloud Storage bucket

def gcs_create_bucket(bucket_name, location='us-central1', storage_class="STANDARD"):
    try:
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        bucket.storage_class = storage_class # STANDARD, NEARLINE, COLDLINE, ARCHIVE
        bucket.create(location=location) # Location can be 'us', 'us-central1', 'us-east4'
        print('[ INFO ] Successfully created {} at {}'.format(bucket_name, location))
    except Exception as e:
        print('[ EXCEPTION ] {}'.format(e))

gcs_create_bucket(GCS_BUCKET, GCS_LOCATION)

[ INFO ] Successfully created bqml_vertex at us-central1


In [42]:
# Export BQML Model to the newly created Cloud Storage Bucket

QUERY = f'''
EXPORT MODEL `{GCP_PROJECT_ID}.{BIGQUERY_DATASET}.{BQ_MODEL_NAME}`
OPTIONS(URI = 'gs://{GCS_BUCKET}/model-assets/')
'''
query_job = bq_client.query(QUERY)
rows = query_job.result()

In [ ]:
# Upload Model (that has been exported to Google Cloud Storage) into Vertex AI

def upload_model(
    project: str,
    location: str,
    display_name: str,
    serving_container_image_uri: str,
    artifact_uri: Optional[str] = None,
    serving_container_predict_route: Optional[str] = None,
    serving_container_health_route: Optional[str] = None,
    description: Optional[str] = None,
    serving_container_command: Optional[Sequence[str]] = None,
    serving_container_args: Optional[Sequence[str]] = None,
    serving_container_environment_variables: Optional[Dict[str, str]] = None,
    serving_container_ports: Optional[Sequence[int]] = None,
    instance_schema_uri: Optional[str] = None,
    parameters_schema_uri: Optional[str] = None,
    prediction_schema_uri: Optional[str] = None,
    explanation_metadata: Optional[explain.ExplanationMetadata] = None,
    explanation_parameters: Optional[explain.ExplanationParameters] = None,
    sync: bool = True,
):
    
    aiplatform.init(project=project, location=location)
    
    model = aiplatform.Model.upload(
        display_name=display_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=serving_container_image_uri,
        serving_container_predict_route=serving_container_predict_route,
        serving_container_health_route=serving_container_health_route,
        instance_schema_uri=instance_schema_uri,
        parameters_schema_uri=parameters_schema_uri,
        prediction_schema_uri=prediction_schema_uri,
        description=description,
        serving_container_command=serving_container_command,
        serving_container_args=serving_container_args,
        serving_container_environment_variables=serving_container_environment_variables,
        serving_container_ports=serving_container_ports,
        explanation_metadata=explanation_metadata,
        explanation_parameters=explanation_parameters,
        sync=sync,
    )
    
    model.wait()
    
    print(model.display_name)
    print(model.resource_name)
    return model



In [67]:
# Pre-built Model Serving Containers:
# https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers

aiplatform.init(project=GCP_PROJECT_ID, location=VERTEX_LOCATION)
    
model = aiplatform.Model.upload(
    display_name=VERTEX_MODEL_NAME,
    artifact_uri=f'gs://{GCS_BUCKET}/model-assets/',
    serving_container_image_uri=VERTEX_SERVING_CONTAINER
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/492854496121/locations/us-central1/models/530307652215898112/operations/8475334968938070016
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/492854496121/locations/us-central1/models/530307652215898112
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/492854496121/locations/us-central1/models/530307652215898112')


In [68]:
# Deploy Vertex Model as an Endpoint

endpoint = model.deploy(
    deployed_model_display_name=f'''{VERTEX_MODEL_NAME}-endpoint''',
    machine_type="n1-standard-4",
    min_replica_count=1,
    max_replica_count=3
)


INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/492854496121/locations/us-central1/endpoints/1907890168702959616/operations/2616151853729054720
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/492854496121/locations/us-central1/endpoints/1907890168702959616
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/492854496121/locations/us-central1/endpoints/1907890168702959616')
INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/492854496121/locations/us-central1/endpoints/1907890168702959616
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/492854496121/locations/us-central1/endpoints/1907890168702959616/operations/1004707612060549120
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/49285449612

## **Get Predictions**

In [70]:
import requests

GCP_PROJECT_ID      = GCP_PROJECT_ID  # 'dz-apps'
VERTEX_REGION       = VERTEX_LOCATION # 'us-central1'
VERTEX_ENDPOINT_ID  = '1907890168702959616'
ACCESS_TOKEN        = 'ya29.a0AfH6SMAnM8jdDzbxNJb3LyfArR0-k41L3sKxaizAlnbhe5ui-3ggO9A8cZLqY4JDNQ3Z1gBsZhIseTUntLb08WAH8hEL6zatM-9-JW0EM8VmPEUKCvPhOjm1UNytrQ07tYaF4kYL1D1G3Rs-Og4K-ff-kXrOK7ec_nWZ9gmUzXPUX6N05hI6QhITbhNLKGKqOl8fq2G2LlGMO5oLIUcBUCPoRVfydMX9hllrWlCGUiwJTfZaJ-VUr_Ladl_k-9Jk1preNrc'  # $(gcloud auth print-access-token)
HEADER = {"Authorization": f"Bearer {ACCESS_TOKEN}", "Content-Type": "application/json"}

PAYLOAD = {
  "instances": [
    {"age": 39,
    "bill_amt_1": 47174,
    "bill_amt_2": 47974,
    "bill_amt_3": 48630,
    "bill_amt_4": 50803,
    "bill_amt_5": 30789,
    "bill_amt_6": 15874,
    "education_level": "1",
    "limit_balance": 50000,
    "marital_status": "2",
    "pay_0": 0,
    "pay_2":0,
    "pay_3": 0,
    "pay_4": 0,
    "pay_5": "0",
    "pay_6": "0",
    "pay_amt_1": 1800,
    "pay_amt_2": 2000,
    "pay_amt_3": 3000,
    "pay_amt_4": 2000,
    "pay_amt_5": 2000,
    "pay_amt_6": 2000,
    "sex": "1"}
  ]
}

response = requests.post(f'https://{VERTEX_REGION}-prediction-aiplatform.googleapis.com/v1alpha1/projects/{GCP_PROJECT_ID}/locations/{VERTEX_REGION}/endpoints/{VERTEX_ENDPOINT_ID}:predict', json=PAYLOAD, HEADER=HEADER)

print('{} - {}'.format(response.status_code, response.content))

TypeError: request() got an unexpected keyword argument 'HEADER'